In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
from zipfile import ZipFile 

with ZipFile('/content/gdrive/My Drive/nerve.zip', 'r') as zip: 
    print('Extracting all the files now...') 
    zip.extractall() 
    print('Done!') 

Extracting all the files now...
Done!


In [3]:
src = '/content/nerve/data/model'
dst = '/content/model'

from distutils.dir_util import copy_tree
copy_tree(src, dst)

['/content/model/u_net.py',
 '/content/model/README.md',
 '/content/model/__pycache__/__init__.cpython-36.pyc',
 '/content/model/__pycache__/u_net.cpython-36.pyc',
 '/content/model/__pycache__/losses.cpython-36.pyc',
 '/content/model/losses.py',
 '/content/model/__init__.py']

In [4]:
import os
import cv2
import shutil
import numpy as np
import pandas as pd
from tqdm import tqdm
from model.u_net import get_unet_256
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

Using TensorFlow backend.


In [0]:
try:
    os.mkdir('/content/img_mask')
except:
    pass

for i in os.listdir('/content/nerve/data/train'):
    if 'mask' in i:
        src = os.path.join('/content/nerve/data/train', i)
        dst = os.path.join('/content', 'img_mask', i)
        shutil.move(src, dst)

for i in os.listdir(os.path.join('/content', 'img_mask')):
    src = os.path.join('/content', 'img_mask', i)
    dst = os.path.join('/content/img_mask', '_'.join(i.split('_')[:2]) + '.tif')
    os.rename(src, dst)

In [0]:
data_dir = '/content'

In [0]:
input_size = 256
epochs = 20
batch_size = 16

orig_width = 580
orig_height = 420

threshold = 0.5

In [0]:
model = get_unet_256()

In [9]:
df_train = pd.read_csv('/content/nerve/data/labels/train_masks.csv')

df_train['img'] = df_train['subject'].astype(str) + '_' + df_train['img'].astype(str)
df_train.head()

,subject,img,pixels
0,1,1_1,168153 9 168570 15 168984 22 169401 26 169818 ...
1,1,1_2,96346 7 96762 16 97179 27 97595 35 98012 40 98...
2,1,1_3,121957 2 122365 26 122784 28 123202 31 123618 ...
3,1,1_4,110980 9 111393 21 111810 33 112228 43 112647 ...
4,1,1_5,NaN


In [0]:
ids_train = df_train['img']

In [11]:
ids_train_split, ids_valid_split = train_test_split(ids_train, test_size=0.1, random_state=42)

print('Training on {} samples'.format(len(ids_train_split)))
print('Validating on {} samples'.format(len(ids_valid_split)))

Training on 5071 samples
Validating on 564 samples


In [0]:
def randomHueSaturationValue(image, hue_shift_limit=(-180, 180),
                             sat_shift_limit=(-255, 255),
                             val_shift_limit=(-255, 255), u=0.5):
    if np.random.random() < u:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(image)
        hue_shift = np.random.uniform(hue_shift_limit[0], hue_shift_limit[1])
        h = cv2.add(h, hue_shift)
        sat_shift = np.random.uniform(sat_shift_limit[0], sat_shift_limit[1])
        s = cv2.add(s, sat_shift)
        val_shift = np.random.uniform(val_shift_limit[0], val_shift_limit[1])
        v = cv2.add(v, val_shift)
        image = cv2.merge((h, s, v))
        image = cv2.cvtColor(image, cv2.COLOR_HSV2BGR)

    return image


def randomShiftScaleRotate(image, mask,
                           shift_limit=(-0.0625, 0.0625),
                           scale_limit=(-0.1, 0.1),
                           rotate_limit=(-45, 45), aspect_limit=(0, 0),
                           borderMode=cv2.BORDER_CONSTANT, u=0.5):
    if np.random.random() < u:
        height, width, channel = image.shape

        angle = np.random.uniform(rotate_limit[0], rotate_limit[1])  # degree
        scale = np.random.uniform(1 + scale_limit[0], 1 + scale_limit[1])
        aspect = np.random.uniform(1 + aspect_limit[0], 1 + aspect_limit[1])
        sx = scale * aspect / (aspect ** 0.5)
        sy = scale / (aspect ** 0.5)
        dx = round(np.random.uniform(shift_limit[0], shift_limit[1]) * width)
        dy = round(np.random.uniform(shift_limit[0], shift_limit[1]) * height)

        cc = np.math.cos(angle / 180 * np.math.pi) * sx
        ss = np.math.sin(angle / 180 * np.math.pi) * sy
        rotate_matrix = np.array([[cc, -ss], [ss, cc]])

        box0 = np.array([[0, 0], [width, 0], [width, height], [0, height], ])
        box1 = box0 - np.array([width / 2, height / 2])
        box1 = np.dot(box1, rotate_matrix.T) + np.array([width / 2 + dx, height / 2 + dy])

        box0 = box0.astype(np.float32)
        box1 = box1.astype(np.float32)
        mat = cv2.getPerspectiveTransform(box0, box1)
        image = cv2.warpPerspective(image, mat, (width, height), flags=cv2.INTER_LINEAR, borderMode=borderMode,
                                    borderValue=(
                                        0, 0,
                                        0,))
        mask = cv2.warpPerspective(mask, mat, (width, height), flags=cv2.INTER_LINEAR, borderMode=borderMode,
                                   borderValue=(
                                       0, 0,
                                       0,))

    return image, mask


def randomHorizontalFlip(image, mask, u=0.5):
    if np.random.random() < u:
        image = cv2.flip(image, 1)
        mask = cv2.flip(mask, 1)

    return image, mask

In [0]:
def train_generator():
    
    while True:
        for start in range(0, len(ids_train_split), batch_size):
            
            x_batch = []
            y_batch = []
            end = min(start + batch_size, len(ids_train_split))
            ids_train_batch = ids_train_split[start:end]
            
            for id in ids_train_batch.values:
                img = cv2.imread(os.path.join(data_dir, 'nerve/data/train/{}.tif'.format(id)))
                img = cv2.resize(img, (input_size, input_size))
                mask = cv2.imread(os.path.join(data_dir, 'img_mask/{}.tif'.format(id)), cv2.IMREAD_GRAYSCALE)
                mask = cv2.resize(mask, (input_size, input_size))
                
                img = randomHueSaturationValue(img,
                                               hue_shift_limit=(-50, 50),
                                               sat_shift_limit=(-5, 5),
                                               val_shift_limit=(-15, 15))
                img, mask = randomShiftScaleRotate(img, mask,
                                                   shift_limit=(-0.0625, 0.0625),
                                                   scale_limit=(-0.1, 0.1),
                                                   rotate_limit=(-0, 0))
                img, mask = randomHorizontalFlip(img, mask)
                
                mask = np.expand_dims(mask, axis=2)
                x_batch.append(img)
                y_batch.append(mask)
                
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            
            yield x_batch, y_batch

In [0]:
def valid_generator():
    
    while True:
        for start in range(0, len(ids_valid_split), batch_size):
            
            x_batch = []
            y_batch = []
            end = min(start + batch_size, len(ids_valid_split))
            ids_valid_batch = ids_valid_split[start:end]
            
            for id in ids_valid_batch.values:
                img = cv2.imread(os.path.join(data_dir, 'nerve/data/train/{}.tif'.format(id)))
                img = cv2.resize(img, (input_size, input_size))
                mask = cv2.imread(os.path.join(data_dir, 'img_mask/{}.tif'.format(id)), cv2.IMREAD_GRAYSCALE)
                mask = cv2.resize(mask, (input_size, input_size))
                mask = np.expand_dims(mask, axis=2)
                x_batch.append(img)
                y_batch.append(mask)
                
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            
            yield x_batch, y_batch

In [15]:
callbacks = [EarlyStopping(monitor='val_loss', patience=3, verbose=1),
             ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1),
             ModelCheckpoint(monitor='val_loss', filepath='nerve_weights.h5', save_best_only=True,
                             save_weights_only=True)]

model.fit_generator(generator=train_generator(),
                    steps_per_epoch=np.ceil(float(len(ids_train_split)) / float(batch_size)),
                    epochs=epochs,
                    verbose=1,
                    callbacks=callbacks,
                    validation_data=valid_generator(),
                    validation_steps=np.ceil(float(len(ids_valid_split)) / float(batch_size)))



Epoch 1/20
317/317 [==============================] - 212s 669ms/step - loss: 1.2398 - dice_coeff: 0.0606 - val_loss: 1.1685 - val_dice_coeff: 0.0948
Epoch 2/20
317/317 [==============================] - 187s 588ms/step - loss: 1.0859 - dice_coeff: 0.0988 - val_loss: 1.0708 - val_dice_coeff: 0.1265
Epoch 3/20
317/317 [==============================] - 186s 588ms/step - loss: 0.9942 - dice_coeff: 0.1400 - val_loss: 1.0472 - val_dice_coeff: 0.1378
Epoch 4/20
317/317 [==============================] - 186s 588ms/step - loss: 0.9045 - dice_coeff: 0.1935 - val_loss: 0.8818 - val_dice_coeff: 0.1927
Epoch 5/20
317/317 [==============================] - 186s 588ms/step - loss: 0.8148 - dice_coeff: 0.2590 - val_loss: 0.8012 - val_dice_coeff: 0.2574
Epoch 6/20
317/317 [==============================] - 186s 588ms/step - loss: 0.7228 - dice_coeff: 0.3342 - val_loss: 0.6988 - val_dice_coeff: 0.3493
Epoch 7/20
317/317 [==============================] - 187s 589ms/step - loss: 0.6363 - dice_coeff:

In [0]:
ids_test = pd.Series(np.arange(1, 5509))

names = []
for id in ids_test:
    names.append('{}.tif'.format(id))

In [19]:
def run_length_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.transpose().flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

rles = []

model.load_weights(filepath='/content/nerve_weights.h5')

print('Predicting on {} samples with batch_size = {}s'.format(len(ids_test), batch_size))

for start in tqdm(range(0, len(ids_test), batch_size)):
  
    x_batch = []
    end = min(start + batch_size, len(ids_test))
    ids_test_batch = ids_test[start:end]
    
    for id in ids_test_batch.values:
        img = cv2.imread('/content/nerve/data/test/test/{}.tif'.format(id))
        img = cv2.resize(img, (input_size, input_size))
        x_batch.append(img)
        
    x_batch = np.array(x_batch, np.float32) / 255
    preds = model.predict_on_batch(x_batch)
    preds = np.squeeze(preds, axis=3)
    
    for pred in preds:
        prob = cv2.resize(pred, (orig_width, orig_height))
        mask = prob > threshold
        rle = run_length_encode(mask)
        rles.append(rle)

  0%|          | 0/345 [00:00<?, ?it/s]

Predicting on 5508 samples with batch_size = 16...


100%|██████████| 345/345 [01:33<00:00,  3.70it/s]


In [0]:
names = [i.split('.')[0] for i in names]
df = pd.DataFrame({'img': names, 'pixels': rles})
df.to_csv('submission.csv', index=False)